In [16]:
# Setup
import pandas as pd
import numpy as np

#Parameter Lists
#Set the variable as None to ignore it when searching
#Change the value here and run all the cells to get an output
LOCATION = None
ROOM = None
QUARTER_INDEX = None          # A number, refer to the file or dataframe
QUARTER = None                # e.g. "FA13"/"WI19"/"SP15"
COURSE_ID = "CSE"             # e.g. "COGS", "BILD"
COURSE_NUM = ["140"]  # Must be a list of string (double quoted or single quoted)! e.g. ["100", "20A", "120A"]
SECTION = None               # Must be a string! e.g "767325"
INSTRUCTOR_FIRST = "Chung Kuan"    # instructor's first name
INSTRUCTOR_LAST = "Cheng"     # instructor's last name
LOWEST_SIZE = None            # Include only sections that have student number >= this variable
LOWEST_RESPONSE = None        # Include only Capes Questions that have number of response >= this variable
CAPES = [7, 9, 14, 15]        # Must be a list of integer! Include only Capes question with an ID that is in the list. From 5 to 27, plus 51
FILENAME = "CAPE_Chung Kuan Cheng.csv"    # name of output file

df = pd.read_csv("CAPES.csv")
print(len(df))

In [17]:
# Read CSV
dtypes = {'LOCATION':str, 'ROOM':str, 'QUARTER_INDEX':int, 'QUATER':int, 'COURSE_ID':str, 'COURSE_NUM':str, 
          'SECTION':str, 'INSTRUCTOR_LAST':str, 'INSTRUCTOR_FIRST': str, 'CAPE_ID':int, 'QUESTION':str, 
          'SECTION_SIZE':int, 'RESPONSE_SIZE':int, "RATING":float}
df = pd.read_csv("CAPES.csv", usecols = ["LOCATION", "ROOM", "QUARTER_INDEX", "QUARTER", 
                                         "COURSE_ID", "COURSE_NUM", "SECTION", "INSTRUCTOR_LAST", 
                                         "INSTRUCTOR_FIRST", "CAPE_ID", "QUESTION", 
                                         "SECTION_SIZE", "RESPONSE_SIZE", "RATING"],
                 dtype = dtypes)

# clean out empty spaces in cells
df["INSTRUCTOR_FIRST"] = df["INSTRUCTOR_FIRST"].str.strip()
df["INSTRUCTOR_LAST"] = df["INSTRUCTOR_LAST"].str.strip()

def ClassExtractor(LOCATION, ROOM, QUARTER, COURSE_ID, COURSE_NUM, SECTION, INSTRUCTOR_LAST, 
                   INSTRUCTOR_FIRST, LOWEST_SIZE, LOWEST_RESPONSE, CAPES):
    result = df
    
    if LOCATION is not None:
        result = result[result["LOCATION"] == LOCATION]
    
    if ROOM is not None:
        reulst = result[result["ROOM"] == ROOM]
        
    if QUARTER is not None:
        result = result[result["QUATER"] == QUARTER]
    
    if COURSE_ID is not None:
        result = result[result["COURSE_ID"] == COURSE_ID]
        
    if COURSE_NUM is not None:
        result = result[result["COURSE_NUM"].isin(COURSE_NUM)]
        
    if SECTION is not None:
        result = result[result["SECTION"] == SECTION]
    
    if INSTRUCTOR_LAST is not None:
        result = result[result["INSTRUCTOR_LAST"] == INSTRUCTOR_LAST]
        
    if INSTRUCTOR_FIRST is not None:
        result = result[result["INSTRUCTOR_FIRST"] == INSTRUCTOR_FIRST]
        
    if LOWEST_SIZE is not None:
        result = result[result["SECTION_SIZE"] >= LOWEST_SIZE]
        
    if LOWEST_RESPONSE is not None:
        result = result[result["RESPONSE_SIZE"] >= LOWEST_RESPONSE]
        
    if isinstance(CAPES, list):
        result = result[result["CAPE_ID"].isin(CAPES)]
        
    return result

def AlltimeAvg(INSTRUCTOR_FIRST, INSTRUCTOR_LAST, CAPES):
    newdf = (df.loc[df["INSTRUCTOR_LAST"] == INSTRUCTOR_LAST]).loc[df["INSTRUCTOR_FIRST"] == INSTRUCTOR_FIRST]
    dic = {}

    for i in CAPES:
        temp = newdf.loc[newdf["CAPE_ID"] == i]
        dic[i] = temp["RATING"].mean()

    print("All Time Average Rating for", INSTRUCTOR_LAST + ",", INSTRUCTOR_FIRST, "in (CAPE_ID, AVG)")
    for key, value in dic.items():
        print(key, value)

def transpose(output):
    hall = output
    seven_rating = list(hall[hall.CAPE_ID == 7].RATING)
    nine_rating = list(hall[hall.CAPE_ID == 9].RATING)
    fourteen_rating = list(hall[hall.CAPE_ID == 14].RATING)
    fifteen_rating = list(hall[hall.CAPE_ID == 15].RATING)
    df = hall
    for i in hall.SECTION.unique():
        df = df.drop(df[(df['SECTION'] == i) & (df['CAPE_ID'] != 7)].index)

    df['Question_7'] = seven_rating
    df['Question_9'] = nine_rating
    df['Question_14'] = fourteen_rating
    df['Question_15'] = fifteen_rating
    df = df.drop(columns=['QUESTION','CAPE_ID'])
    return df

In [18]:
# Calling Function and Write Data
output = ClassExtractor(LOCATION, ROOM, QUARTER, COURSE_ID, COURSE_NUM, SECTION, INSTRUCTOR_LAST, 
                   INSTRUCTOR_FIRST, LOWEST_SIZE, LOWEST_RESPONSE, CAPES)
output = transpose(output)
filename = "CAPE_" + (INSTRUCTOR_FIRST or "") + " " + (INSTRUCTOR_LAST or "") + ".csv"

if not output.empty:
    output.to_csv(FILENAME or ("CAPE_" + (INSTRUCTOR_FIRST or "") + " " + (INSTRUCTOR_LAST or "") + ".csv"))

if None in (INSTRUCTOR_FIRST, INSTRUCTOR_LAST):
    INSTRUCTOR_FIRST = output["INSTRUCTOR_FIRST"].iloc[0]
    INSTRUCTOR_LAST = output["INSTRUCTOR_LAST"].iloc[0]

if CAPES is None:
    CAPES = list(range(5, 28))
    CAPES.append(51)
    
AlltimeAvg(INSTRUCTOR_FIRST, INSTRUCTOR_LAST, CAPES)
# An summary of overall capes rating of this professor will be display below after running this cell

All Time Average Rating for Cheng, Chung Kuan in (CAPE_ID, AVG)
7 3.0168681915555555
9 2.979205140666666
14 4.25812438
15 4.105578765333333


,LOCATION,ROOM,QUARTER_INDEX,QUARTER,COURSE_ID,COURSE_NUM,SECTION,INSTRUCTOR_LAST,INSTRUCTOR_FIRST,SECTION_SIZE,RESPONSE_SIZE,RATING,Question_7,Question_9,Question_14,Question_15
133560,EBU3B,B230,56,FA15,CSE,8A,849741,Cao,Yingjun,46,40,3.900000,3.900000,3.925000,4.512821,4.512821
133581,EBU3B,B230,56,FA15,CSE,8A,849742,Cao,Yingjun,49,41,3.804878,3.804878,3.682927,4.439024,4.325000
133602,EBU3B,B230,56,FA15,CSE,8A,849743,Cao,Yingjun,48,39,4.025641,4.025641,4.025641,4.435897,4.631579
133623,EBU3B,B230,56,FA15,CSE,8A,849746,Cao,Yingjun,48,45,4.511111,4.511111,4.355556,4.622222,4.568182
133644,EBU3B,B230,56,FA15,CSE,8A,849747,Cao,Yingjun,47,38,4.236842,4.236842,4.184211,4.447368,4.421053
133665,EBU3B,B230,56,FA15,CSE,8A,849748,Cao,Yingjun,50,42,4.261905,4.261905,3.975610,4.523810,4.523810
189033,EBU3B,B230,79,FA16,CSE,8A,882229,Cao,Yingjun,62,59,4.186441,4.186441,3.966102,4.694915,4.724138
189054,EBU3B,B230,79,FA16,CSE,8A,882230,Cao,Yingjun,61,55,4.290909,4.290909,4.163636,4.563636,4.600000
189075,EBU3B,B230,79,FA16,CSE,8A,882231,Cao,Yingjun,63,59,4.474576,4.474576,4.157895,4.728814,4.758621
189096,EBU3B,B230,79,FA16,CSE,8A,882232,Cao,Yingjun,60,59,4.508475,4.508475,4.288136,4.677966,4.745763
